# Train ESCNN $C_d$ Model

This Jupyter Notebook is designed to train and evaluate an Element Spatial Convolutional Neural Network (ESCNN) model for predicting the drag coefficient ($C_d$) of airfoils. The workflow includes data preprocessing, model training, and evaluation. The notebook leverages PyTorch for building and training the neural network, and various Python libraries for data manipulation and visualization.

### Workflow:
1. **Data Preparation**:
    - Load and preprocess airfoil coordinate and polar data.
    - Downsample the coordinates to a fixed size.
    - Organize the data into a suitable format for the neural network.

2. **Model Definition**:
    - Define the ESCNN model architecture for predicting $C_d$.
    - Define a custom loss function that incorporates both data-driven and physics-informed components.

3. **Training**:
    - Split the data into training and validation sets.
    - Train the ESCNN model using the training data.
    - Monitor the training and validation loss over epochs.

4. **Evaluation**:
    - Evaluate the trained model on a separate test dataset.
    - Compute evaluation metrics such as relative L2 norm and R² score.
    - Save and visualize the evaluation results.

5. **Results Visualization**:
    - Plot the training and validation loss curves.
    - Optionally save the trained model and evaluation results.

This notebook provides a comprehensive approach to developing a physics-informed neural network model for aerodynamic predictions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
import re

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
import time


In [ ]:
SAVE_TRAINED_MODEL = False
PLOT_RESULTS = False
SAVE_RESULTS = False

project_path = '/mnt/e/eVTOL_model/eVTOL-VehicleModel/'
data_path = project_path + 'data/airfoil_data/'
model_path = project_path + 'trained_models/models/airfoil/'
save_path = project_path + 'result/airfoil_model/'

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import datetime
date = datetime.date.today()

In [ ]:
# Downsampling using average pooling
"""
Downsample the input array to 35 elements using interpolation.
"""

def downsample_to_35(input_array):
    input_tensor = torch.tensor(input_array, dtype=torch.float32)
    
    # Reshape the input to be 1D (if it's not already)
    if input_tensor.dim() == 1:
        input_tensor = input_tensor.unsqueeze(0).unsqueeze(0)  # Shape (1, 1, original_length)
    elif input_tensor.dim() == 2:
        input_tensor = input_tensor.unsqueeze(0)  # Shape (1, original_channels, original_length)
    
    # Perform interpolation to downsample to 35 elements
    downsampled_tensor = F.interpolate(input_tensor, size=35, mode='linear', align_corners=True)
    
    # Remove the unnecessary dimensions to return a 1D tensor
    downsampled_array = downsampled_tensor.squeeze().numpy()
    
    return downsampled_array

In [ ]:

def prep_data(root, keyword=None):

    airfoil_count = 0
    # Initialization of arrays
    # Coordinates
    x_i = [] # Initial coordinates - before downsizing
    y_i = []

    # Polars
    alphas = []
    Cls = []
    Cds = []
    Cms = []

    if keyword:
        # lists of files in each dir
        coord_files = [f for f in os.listdir(root) if f == (keyword+'_coordinates.dat')]
        polar_files = [f for f in os.listdir(root) if f == ('xf-'+keyword+'-il-1000000.csv')]
    else:
        # lists of files in each dir
        coord_files = [f for f in os.listdir(root) if f.endswith('_coordinates.dat')]
        polar_files = [f for f in os.listdir(root) if f.endswith('.csv')]

    # Extract base names from coordinate files
    coord_bases = {re.sub(r'\_coordinates.dat$', '', f) for f in coord_files}
    polar_bases = {}
    for polar_file in polar_files:
        match = re.match(r'xf-(.*)-il-1000000\.csv$', polar_file)
        if match:
            base_name = match.group(1)
            polar_bases[base_name] = polar_file
    # print(polar_bases)
    for base_name in coord_bases:
        if base_name in polar_bases:
            coord_file = f"{base_name}_coordinates.dat"
            polar_file = polar_bases[base_name]

            coordinate_data = np.loadtxt(root+coord_file)
            # polar_data = np.loadtxt(root+polar_file, skiprows=12)
            polar_data = pd.read_csv(root+polar_file, skiprows=10)
            polar_data = polar_data[(polar_data['Alpha'] >= -2) & (polar_data['Alpha'] <= 10)]
            # print(len(polar_data))

            # Coordinates
            x = []
            y = []

            # Polars
            alpha = polar_data['Alpha'].values
            Cl = polar_data['Cl'].values
            Cd = polar_data['Cd'].values
            Cm = polar_data['Cm'].values

            # print(alpha)

            for i in range(0, len(coordinate_data)):
                np.array(x.append(float(coordinate_data[i][0]))) 
                np.array(y.append(float(coordinate_data[i][1])))
                
            if len(x) >= 35:    # Only consider the files with more than 35 coordinates
                x_i.append(x)
                y_i.append(y)

                alphas.append(alpha)

                for num_val in range(len(Cl)):
                    Cls.append(Cl[num_val])
                    Cds.append(Cd[num_val])
                    Cms.append(Cm[num_val])

                airfoil_count += 1

    print(f"Number of airfoils: {airfoil_count}")
                        
    return x_i, y_i, Cls, Cds, Cms, alphas
       

In [ ]:

root_train = data_path + "/training_database/"
x_i, y_i, Cls, Cds, Cms, alphas = prep_data(root_train)
Cls = np.array(Cls, dtype=float)
Cds = np.array(Cds, dtype=float)


In [ ]:
# Downsample to 35 elements
x_f = [] # Final coordinates - after downsizing
y_f = []
for num_airfoil in range(0, len(x_i)):
    downsampled_x = downsample_to_35(x_i[num_airfoil])
    downsampled_y = downsample_to_35(y_i[num_airfoil])

    x_f.append(downsampled_x)
    y_f.append(downsampled_y)
    

## Arrange the data in the form of elements
* $E = [E_1, E_2, ....., E_n]$ 
* $E_1 = [x_1, y_1, x_2, y_2, \alpha]$

In [ ]:
# Arange the input data in columns [x, y, alpha, Re, M]

def organize_data(x_f, y_f, alphas):

    Elements = []

    # Loop through the polars
    for n_file in range(len(x_f)):
        x_temp = x_f[n_file]
        y_temp = y_f[n_file]
        alpha_temp = alphas[n_file]
        
        for j in range(len(alpha_temp)):
            batch = []
            # Loop through the coodrinates
            for i in range(len(x_temp)-1):
                element = np.array([x_temp[i], y_temp[i], x_temp[i+1], y_temp[i+1], alpha_temp[j]])
                # Elements.append(element)
                batch.append(element)
            batch = np.array(batch)
            batch = batch.flatten()
            Elements.append(batch)

    Elements = np.array(Elements)

    return Elements


In [ ]:
Elements = organize_data(x_f, y_f, alphas)
print(Elements.shape)

## Neural Network Model

In [ ]:
# Element Spatial Convolutional Neural Network model

class ESCNN(nn.Module):
    def __init__(self):
        super(ESCNN, self).__init__()
        
        # Conv1: Assume 1D Convolution
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=200, kernel_size=5, stride=5)
        self.relu1 = nn.ReLU()

        #conv2
        self.conv2 = nn.Conv1d(in_channels=200, out_channels=200, kernel_size=1)
        self.relu2 = nn.ReLU()
        
        # Conv2
        self.conv3 = nn.Conv1d(in_channels=200, out_channels=1, kernel_size=5, padding=2)
        self.relu3 = nn.ReLU()
        
        # Final fully connected layer to output scalar
        self.fc1 = nn.Linear(in_features=34, out_features=34)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(in_features=34, out_features=1)

        # Define learnable parameters for C_d0 and k
        self.Cd0 = nn.Parameter(torch.tensor(0.02))  # Initialized zero-lift drag coefficient
        self.k = nn.Parameter(torch.tensor(0.05))    # Initialized induced drag factor

    
    def forward(self, x):
        # Reshape input if necessary, ensure it's in the shape (batch_size, channels, elements)
        x = x.view(-1, 1, 170)  # Reshape to (batch_size, channel=1, elements=170)
        
        x = self.conv1(x)  
        x = self.relu1(x)
        
        x = self.conv2(x)  
        x = self.relu2(x)

        x = self.conv3(x)  
        x = self.relu3(x)
        
        
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)  # (batch_size, 1)
        x = self.relu4(x)

        x = self.fc2(x)
        
        return x

model = ESCNN()

model.to(device)


Orgaize the data for the NN

In [ ]:
input_data = Elements
output_data = Cds
print(output_data.shape)


input_data_train, input_data_val, output_data_train, output_data_val = train_test_split(input_data, output_data, test_size=0.25, random_state=28)


In [ ]:
# Convert to PyTorch tensors
# And move the data to cuda

input_data_train = torch.from_numpy(input_data_train).float().to(device)
input_data_val = torch.from_numpy(input_data_val).float().to(device)

output_data_train = torch.from_numpy(output_data_train).float().to(device)
output_data_val = torch.from_numpy(output_data_val).float().to(device)

In [ ]:
# DataLoader
dataset_train = TensorDataset(input_data_train, output_data_train)
dataset_val = TensorDataset(input_data_val, output_data_val)

trainDataLoader = DataLoader(dataset_train, batch_size=128)
valDataLoader = DataLoader(dataset_val, batch_size=128)


## Train the Model

In [ ]:
def custom_loss(pred_cd, true_cd, cl_pred, model, criterion):
    # MSE loss between predicted and true Cd
    mse_loss = criterion(pred_cd, true_cd)
    
    # Physics-informed loss using the model's learnable Cd0 and k
    phy_cd = model.Cd0 + model.k * cl_pred ** 2
    drag_polar_loss = criterion(pred_cd, phy_cd)

    # Total loss
    total_loss = mse_loss + 0.3 * drag_polar_loss  # Adjust weight (0.1) as needed

    return total_loss


In [ ]:
class ESCNN_Cl(nn.Module):
    def __init__(self):
        super(ESCNN_Cl, self).__init__()
        
        # Conv1: Assume 1D Convolution
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=200, kernel_size=5, stride=5)
        self.relu1 = nn.ReLU()

        #conv2
        self.conv2 = nn.Conv1d(in_channels=200, out_channels=200, kernel_size=1)
        self.relu2 = nn.ReLU()

        # Conv3
        self.conv3 = nn.Conv1d(in_channels=200, out_channels=100, kernel_size=1)
        self.relu3 = nn.ReLU()
        
        # Conv4
        self.conv4 = nn.Conv1d(in_channels=100, out_channels=1, kernel_size=5, padding=2)
        self.relu4 = nn.ReLU()
        
        # Final fully connected layer to output scalar
        self.fc1 = nn.Linear(in_features=34, out_features=34)
        self.relu5 = nn.ReLU()

        self.fc2 = nn.Linear(in_features=34, out_features=1)
    
    def forward(self, x):
        # Reshape input if necessary, ensure it's in the shape (batch_size, channels, elements)
        x = x.view(-1, 1, 170)  # Reshape to (batch_size, channel=1, elements=170)
        
        x = self.conv1(x)  
        x = self.relu1(x)
        
        x = self.conv2(x)  
        x = self.relu2(x)

        x = self.conv3(x)  
        x = self.relu3(x)
        
        x = self.conv4(x)  
        x = self.relu4(x)
        
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)  
        x = self.relu5(x)

        x = self.fc2(x)
        
        return x

In [ ]:
# Initialize the model for Cl
cl_model = ESCNN_Cl()
cl_model.load_state_dict(torch.load('/mnt/e/eVTOL_model/eVTOL-AirfoilModel/trained_model/Cl_models/2024-11-18_model_Cl_ESCNN_lr1e-05_e1500_rbf170_convL4.pth'), strict=False)
cl_model = cl_model.to(device)
cl_model.eval()


In [ ]:
# Define loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error Loss
lr = 5e-5
optimizer = optim.Adam(model.parameters(), lr=lr)
num_epochs = 250
num_convL_layer = 3

# initialize a dictionary to store training history
H = {
	"train_loss": [],
	"val_loss": [],
    "Cd0_train": [],
    "Cd0_val": [],
    "k_train": [],
    "k_val": []

}

# measure how long training is going to take
print("[INFO] training the network...")
startTime = time.time()


# Training loop

for e in range(0, num_epochs):
    model.train()

    totalTrainLoss = 0
    totalValLoss = 0

    for (ip, op) in trainDataLoader:
        pred_cd = model(ip)
        pred_cd = pred_cd.squeeze(1)

        with torch.no_grad():  # Cl model is pre-trained and frozen
            cl_pred = cl_model.forward(ip)
            cl_pred = cl_pred.squeeze(1)

        # Compute the custom loss (data + physics-informed loss)
        loss = custom_loss(pred_cd, op, cl_pred, model, criterion)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        totalTrainLoss += loss.item()

     # Store the values of Cd0 and k after each epoch
    H["Cd0_train"].append(model.Cd0.item())
    H["k_train"].append(model.k.item())

    with torch.no_grad():
        model.eval()

        for (ip, op) in valDataLoader:
            pred_cd = model(ip)
            pred_cd = pred_cd.squeeze(1)

            cl_pred = cl_model.forward(ip)
            cl_pred = cl_pred.squeeze(1)

            val_loss = custom_loss(pred_cd, op, cl_pred, model, criterion)
            totalValLoss += val_loss.item()

         # Store the values of Cd0 and k after each epoch
        H["Cd0_val"].append(model.Cd0.item())
        H["k_val"].append(model.k.item())

    avgTrainLoss = totalTrainLoss / len(trainDataLoader.dataset)
    avgValLoss = totalValLoss / len(valDataLoader.dataset)

    H["train_loss"].append(avgTrainLoss)
    H["val_loss"].append(avgValLoss)

    print(f"Epoch: {e+1}/{num_epochs}, Train Loss: {avgTrainLoss:.8f}, Val Loss: {avgValLoss:.8f}")

print("Finished Training")


In [ ]:
# Plot the results

plt.figure()
plt.figure(figsize=(10, 6), dpi=300)  # High-resolution plot

plt.plot(np.log10(H["train_loss"]), color='black', linewidth=1.5, label="Training loss")
plt.plot(np.log10(H["val_loss"]), color='red', linewidth=1.5, label="Validation loss")
plt.title("Training Loss and Validation Loss", fontsize=12)
plt.grid(True, linestyle='--', linewidth=0.5)
plt.xlabel("Epoch", fontsize=10)
plt.ylabel("Log(MSE Loss)", fontsize=10)
plt.legend(loc='upper right', fontsize=10)
# plt.gca().set_aspect('equal', adjustable='box')  # Maintain correct aspect ratio
# plt.savefig("airfoil_model_cd_loss_phy_e250.pdf", format="pdf", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
# Save the model if needed
if SAVE_TRAINED_MODEL:
    save_path = model_path + '/{}_model_Cd_ESCNN_lr{}_e{}_convL{}.pth'.format(date, lr, num_epochs,num_convL_layer)

    torch.save(model.state_dict(), save_path)

## Evaluate the model

In [ ]:
from sklearn.metrics import r2_score

root_test = data_path + "/testing_database/"

coord_files = [f for f in os.listdir(root_test) if f.endswith('_coordinates.dat')]
coord_bases = {re.sub(r'\_coordinates.dat$', '', f) for f in coord_files}

def mape(y_true, y_pred):
    """Compute Mean Absolute Percentage Error (MAPE)"""
    mask = y_true != 0  # Avoid division by zero
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def relative_l2_norm(y_true, y_pred):
    """Compute Relative L2 Norm Error (ε)"""
    mask = y_true != 0  # Avoid division by zero
    numerator = np.linalg.norm(y_pred[mask] - y_true[mask], ord=2)  # ||pred - true||_2
    denominator = np.linalg.norm(y_true[mask], ord=2)  # ||true||_2
    return (numerator / denominator) * 100

SAVE_PATH = '/mnt/e/eVTOL_model/eVTOL-VehicleModel/result/airfoil_model/'

# Initialize lists to store evaluation results
l2_norm_cd_list = []
r2_cd_list = []
airfoil_names = []

for keyword in coord_bases:
    x_t, y_t, Cls_t, Cds_t, Cms_t, alphas_t = prep_data(root_test, keyword)
    Cls_t = np.array(Cls_t, dtype=float)
    Cds_t = np.array(Cds_t, dtype=float)
    Cms_t = np.array(Cms_t, dtype=float)
    
    x_f_t = [] # Final coordinates - after downsizing
    y_f_t = []
    for num_airfoil in range(0, len(x_t)):
        downsampled_x = downsample_to_35(x_t[num_airfoil])
        downsampled_y = downsample_to_35(y_t[num_airfoil])

        x_f_t.append(downsampled_x)
        y_f_t.append(downsampled_y)
        
    Elements_t = organize_data(x_f_t, y_f_t, alphas_t)
    # print("Elements: ",Elements_t.shape)
    # print("Cds: ",Cds_t.shape)

    if Elements_t.shape != (0,):
        input_data_test = Elements_t

        # Convert to PyTorch tensors
        input_data_test = torch.tensor(input_data_test, dtype=torch.float32)

        # Move data to GPU
        input_data_test = input_data_test.to(device)

        # Evaluate the model on test dataset
        with torch.no_grad():
            Cd_eval = model.forward(input_data_test)
            # loss = criterion(Cl_eval, Cls_test[0])

        # Calculate the evaluation metrics
        Cd_eval = Cd_eval.cpu().numpy().flatten()
        l2_norm_cd = relative_l2_norm(Cds_t, Cd_eval)
        r2_cd = r2_score(Cds_t, Cd_eval)

        l2_norm_cd_list.append(l2_norm_cd)
        r2_cd_list.append(r2_cd)
        airfoil_names.append(keyword)

        print(f"Airfoil: {keyword}, relative L2 Norm percentage Cd: {l2_norm_cd:.2f}%, R^2 Cd: {r2_cd:.4f}")

        # plt.figure()
        # plt.plot(alphas_t[0], Cd_eval)
        # plt.plot(alphas_t[0],Cds_t)

        # plt.legend(['NN Model', 'UIUC database'])
        # plt.title(r'$C_d$ vs $\alpha$ for {} airfoil'.format(keyword))
        # plt.xlabel(r'AOA [$\alpha$]')
        # plt.ylabel(r'$C_d$')

    else:
        continue

# Save the evaluation results
mean_l2_norm_cd = np.mean(l2_norm_cd_list)
r2_cd_avg = np.mean(r2_cd_list)

print(f"Average MAPE Cd: {mean_l2_norm_cd:.2f}%, Average R^2 Cd: {r2_cd_avg:.4f}")

# Convert results to a DataFrame
results_df = pd.DataFrame({
    "Airfoil": airfoil_names,
    "Relative L2 Norm Cd (%)": l2_norm_cd_list,
    "R² Cd": r2_cd_list
})

# Append mean values to the DataFrame
mean_row = pd.DataFrame({
    "Airfoil": ["Mean"], 
    "Relative L2 Norm Cd (%)": [mean_l2_norm_cd], 
    "R² Cd": [r2_cd_avg]
})

# results_df = pd.concat([results_df, mean_row], ignore_index=True)

# # Define CSV file path
# csv_filename = os.path.join(SAVE_PATH, "Cd_evaluation_results_lambda_0.3.csv")

# # Save to CSV
# results_df.to_csv(csv_filename, index=False)

# print(f"Results (including mean values) saved to {csv_filename}")


